# This Notebook computes the spectrograms of the 60 second audio recordings.

In [ ]:
# Import section
import numpy as np
import matplotlib.pyplot as plt
import librosa
import os
from scipy import signal

In [ ]:
# Compute Spectogram from the audio recording 

In [ ]:
# Expect the sampling rate to be the same for all recordings.
sampling_rate_expected = 44100

# The window size for the spectrogram.
window_size = 1024

# The window overlap for the spectrogram.
window_overlap = 512

# The window used for the spectrogram. ('tukey', 0.25) is the default.
window_type = signal.get_window( ('tukey', 0.25), window_size )
window_description = 'tukey, 0.25'

In [ ]:
cwd=os.getcwd()
# The directory where the .wav files are located. We will read these.
recording_dir = cwd +'/Recordings/'

# The directory where we will write the computed spectrograms as .png image files.
spectrogram_dir = cwd +'/Spectrograms/'

try:
    os.makedirs(spectrogram_dir)
except:
    pass


In [ ]:
# Write spectrogram parameters.
parameters_filename = spectrogram_dir + '0_spectrogram_parameters.txt'
spectogram_param_file = open(parameters_filename,'w')
spectogram_param_file.write('sampling rate = '+ repr(sampling_rate_expected)+'\n')
spectogram_param_file.write('window size = '+ repr(window_size)+'\n')
spectogram_param_file.write('window overlap = '+ repr(window_overlap)+'\n')
spectogram_param_file.write('window overlap = '+ window_description +'\n')
spectogram_param_file.write('log10 of spectrum\n')
spectogram_param_file.close()

In [ ]:
# Get list of recordings.
recording_list = os.listdir(recording_dir);
n_recordings = len(recording_list)
print(n_recordings)

In [ ]:
# Read each recording and compute spectrogram
recording_num = 0
for recording in recording_list:
    recording_num = recording_num + 1
    recording_filename = recording_dir + recording
#     print('recording '+repr(recording_num)+' of '+repr(n_recordings)+': about to read '+recording)
    # Read the audio file. Don't change the sample rate.
    x, sampling_rate = librosa.load(recording_filename,sr=None)
    
    if sampling_rate != sampling_rate_expected:
        print('WRONG SAMPLING RATE: expected='+repr(sampling_rate_expected)+' recording='+repr(sampling_rate))
        break
        
    # Compute the spectrogram.
    w, t, s = signal.spectrogram(x, sampling_rate, window=window_type, noverlap=window_overlap, nperseg=window_size)
    
    # s: matrix with the 2D spectrogram (will be complex).
    # w: vector with the frequencies spacings of the computed DFT.
    # t: vector with the time spacings of the windows.

    # Compute the spectrum of the spectrogram.
    s_spectrum = abs(s);
    
    # Flip it vertically.
    s_spectrum = np.flip(s_spectrum, 0)

    # Compute log_10 of spectrogram.
    s_spectrum_log = np.log10( s_spectrum + np.finfo(float).eps )
    #plt.imshow(s_spectrum_log,cmap='gray')

    # Normalize so values range from 0 to 1.
    s_spectrum_log = (s_spectrum_log - np.amin(s_spectrum_log)) / (np.amax(s_spectrum_log) - np.amin(s_spectrum_log))

    # Save as png file.
    spectrogram_filename = spectrogram_dir + recording +'.png'
    plt.imsave(spectrogram_filename, s_spectrum_log, cmap='gray')

    # Save time and frequency indices.
    spectrograminfo_filename = spectrogram_dir + recording + '_info.txt'
    spectrograminfo_file = open(spectrograminfo_filename,'w')    
    for i in t:
        spectrograminfo_file.write(repr(i)+'\n')
    for i in w:
        spectrograminfo_file.write(repr(i)+'\n')
    spectrograminfo_file.close()